In [1]:
import MDAnalysis as mda
from MDAnalysis.lib.pkdtree import PeriodicKDTree
import numpy as np

box = np.ones(3).astype(np.float32)*100
points = np.random.uniform(low=0,high=1.0,size=(1000,3))*box
points = np.array(points,order='F',dtype=np.float32)
maxdist = 10.0
points

array([[33.229294, 13.656529, 56.061867],
       [23.825771, 23.437624, 28.862265],
       [79.285576, 85.216835, 84.78078 ],
       ...,
       [14.779184, 39.14883 , 60.47615 ],
       [78.36501 , 99.39047 , 16.328688],
       [ 7.393593, 97.84592 , 68.38035 ]], dtype=float32)

In [2]:
def kdtree_distance(points,box,maxdist):
    kdtree = PeriodicKDTree(box,bucket_size=10)
    kdtree.set_coords(points)
    n = 0
    for idx,centers in enumerate(points):
        kdtree.search(centers,maxdist)
        indices = kdtree.get_indices()
        indices.remove(idx)
        distances = mda.lib.distances.distance_array(centers.reshape((1,3)),points[indices],box)
        pairs = list(zip(indices,distances))
        n += len(indices)
    return n//2

In [3]:
#Test
brute_force = mda.lib.distances.distance_array(points,points,box)
row, col = np.where(brute_force < maxdist)
np.testing.assert_equal((len(row)-points.shape[0])/2, kdtree_distance(points,box,maxdist))

In [5]:
#Performance
%timeit mda.lib.distances.distance_array(points,points,box)

36.5 ms ± 608 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%timeit kdtree_distance(points,box,maxdist)

121 ms ± 5.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
